<a href="https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/05_TS_archs_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

created by Ignacio Oguiza - email: oguiza@timeseriesAI.co

# Import libraries 📚

In [ ]:
# # **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
# stable = True # Set to True for latest pip version or False for main branch in GitHub
# !pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [1]:
pip install tsai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from tsai.all import *
from IPython.display import clear_output
my_setup()

os              : Linux-6.1.123+-x86_64-with-glibc2.35
python          : 3.11.13
tsai            : 0.4.1
fastai          : 2.8.2
fastcore        : 1.8.7
torch           : 2.6.0+cu124
device          : cpu
cpu cores       : 1
threads per cpu : 2
RAM             : 12.67 GB
GPU memory      : N/A


# Experiments 🧪

I've run a small test to show how you can build any model in the `tsai` library using the `create_model` functions.

The esiest way to do it is pass the architecture you want to use, a `TSDataLoaders` and any `kwargs` you'd like to use. The `create_model` function will pick up the necessary data from the `TSDataLoaders` object.

I've used 2 multivariate time series datasets. As you can see, it's difficult to predict which architecture will work best for which dataset.

Please, bear in mind that this is a very simple test without any hyperparameter tuning.

In [ ]:

bs = 64
X = np.load("X_normalizes_tsai.npy")
y = np.load("y_int_tsai.npy")
splits = get_splits(y, valid_size=.2, stratify=True, random_state=23, shuffle=True)
splits =
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(HydraMultiRocketPlus,{}),(TransformerRNNPlus, {}),(TransformerGRUPlus, {}),(TransformerLSTMPlus, {}),(XceptionTime, {}),(XceptionTimePlus,{}), (OmniScaleCNN, {}), (mWDN, {'levels': 4}),(mWDNPlus, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'roc_auc', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=RocAucBinary())
    min_grad_lr = learn.recorder.min_grad_lr

    start = time.time()
    learn.fit_one_cycle(50, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='roc_auc', ascending=False, kind='stable', ignore_index=True, inplace=True)
    preds = learn.get_preds()
    preds = preds.cpu().numpy()
    targs = targs.cpu().numpy()

    # Build a descriptive model name
    model_name = arch.__name__
    if 'n_layers' in k or 'bidirectional' in k:
        extra = f"_layers{k.get('n_layers', '')}_bi{k.get('bidirectional', False)}"
        model_name += extra
    elif 'shuffle' in k:
        model_name += '_noshuffle'

    # Save predictions and targets separately
    np.save(f"{model_name}_preds.npy", preds)
    np.save(f"{model_name}_targs.npy", targs)
    clear_output()
    display(results)

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,LSTM_FCN,{},347246,0.074354,0.116819,0.966667,6
1,FCN,{},285446,0.074158,0.125400,0.961111,5
2,LSTM_FCN,{'shuffle': False},336446,0.066596,0.113484,0.961111,6
3,XceptionTime,{},403420,0.399169,0.499384,0.961111,9
4,ResNet,{},490758,0.025244,0.122984,0.955556,7
5,ResCNN,{},268551,0.041432,0.134848,0.950000,6
6,InceptionTime,{},460038,0.026669,0.113258,0.938889,9
7,mWDN,{'levels': 4},467038,0.025454,0.274648,0.927778,9
8,xresnet1d34,{},7232518,0.022142,0.318501,0.916667,15
9,OmniScaleCNN,{},5239596,0.193854,0.260377,0.883333,19
